In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install pytorch-tabnet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/kaggle/input/dairy-dataset/dairy_dataset.csv')

# Preview the dataset
print(df.shape)
print(df.columns)
print(df.head())

# Basic info
df.info()
df.describe()

(4325, 23)
Index(['Location', 'Total Land Area (acres)', 'Number of Cows', 'Farm Size',
       'Date', 'Product ID', 'Product Name', 'Brand', 'Quantity (liters/kg)',
       'Price per Unit', 'Total Value', 'Shelf Life (days)',
       'Storage Condition', 'Production Date', 'Expiration Date',
       'Quantity Sold (liters/kg)', 'Price per Unit (sold)',
       'Approx. Total Revenue(INR)', 'Customer Location', 'Sales Channel',
       'Quantity in Stock (liters/kg)', 'Minimum Stock Threshold (liters/kg)',
       'Reorder Quantity (liters/kg)'],
      dtype='object')
        Location  Total Land Area (acres)  Number of Cows Farm Size  \
0      Telangana                   310.84              96    Medium   
1  Uttar Pradesh                    19.19              44     Large   
2     Tamil Nadu                   581.69              24    Medium   
3      Telangana                   908.00              89     Small   
4    Maharashtra                   861.95              21    Medium   

   

,Total Land Area (acres),Number of Cows,Product ID,Quantity (liters/kg),Price per Unit,Total Value,Shelf Life (days),Quantity Sold (liters/kg),Price per Unit (sold),Approx. Total Revenue(INR),Quantity in Stock (liters/kg),Minimum Stock Threshold (liters/kg),Reorder Quantity (liters/kg)
count,4325.000000,4325.000000,4325.000000,4325.000000,4325.000000,4325.000000,4325.000000,4325.000000,4325.00000,4325.000000,4325.000000,4325.000000,4325.000000
mean,503.483073,54.963699,5.509595,500.652657,54.785938,27357.845411,29.127630,248.095029,54.77914,13580.265401,252.068671,55.826143,109.107820
std,285.935061,26.111487,2.842979,288.975915,26.002815,21621.051594,30.272114,217.024182,26.19279,14617.009122,223.620870,26.301450,51.501035
min,10.170000,10.000000,1.000000,1.170000,10.030000,42.516500,1.000000,1.000000,5.21000,12.540000,0.000000,10.020000,20.020000
25%,252.950000,32.000000,3.000000,254.170000,32.460000,9946.814500,10.000000,69.000000,32.64000,2916.650000,66.000000,32.910000,64.280000
50%,509.170000,55.000000,6.000000,497.550000,54.400000,21869.652900,22.000000,189.000000,54.14000,8394.540000,191.000000,56.460000,108.340000
75%,751.250000,77.000000,8.000000,749.780000,77.460000,40954.441000,30.000000,374.000000,77.46000,19504.550000,387.000000,79.010000,153.390000
max,999.530000,100.000000,10.000000,999.930000,99.990000,99036.369600,150.000000,960.000000,104.51000,89108.900000,976.000000,99.990000,199.950000


In [5]:
# Handle missing values
df = df.dropna()  # Or use imputation if needed

# Parse date columns if available
df['Date'] = pd.to_datetime(df['Date'])
df['Production Date'] = pd.to_datetime(df['Production Date'])
df['Expiration Date'] = pd.to_datetime(df['Expiration Date'])

# Feature: Days Until Expiry
df['Days_Until_Expiry'] = (df['Expiration Date'] - df['Date']).dt.days

# Drop rows where expiry is negative (expired stock)
df = df[df['Days_Until_Expiry'] >= 0]


In [6]:
print(df.columns.tolist())


['Location', 'Total Land Area (acres)', 'Number of Cows', 'Farm Size', 'Date', 'Product ID', 'Product Name', 'Brand', 'Quantity (liters/kg)', 'Price per Unit', 'Total Value', 'Shelf Life (days)', 'Storage Condition', 'Production Date', 'Expiration Date', 'Quantity Sold (liters/kg)', 'Price per Unit (sold)', 'Approx. Total Revenue(INR)', 'Customer Location', 'Sales Channel', 'Quantity in Stock (liters/kg)', 'Minimum Stock Threshold (liters/kg)', 'Reorder Quantity (liters/kg)', 'Days_Until_Expiry']


In [7]:
# Rename for convenience (optional)
df = df.rename(columns={
    'Quantity in Stock (liters/kg)': 'quantity_in_stock',
    'Quantity Sold (liters/kg)': 'quantity_sold',
    'Reorder Quantity (liters/kg)': 'reorder_quantity',
    'Minimum Stock Threshold (liters/kg)': 'min_stock_threshold'
})

# Derived Feature: Demand Velocity
df['demand_velocity'] = df['quantity_sold'] / df['Days_Until_Expiry'].replace(0, 1)

# Target Variables
df['days_to_reorder'] = (df['min_stock_threshold'] - df['quantity_in_stock']) / df['demand_velocity']
df['days_to_reorder'] = df['days_to_reorder'].clip(lower=0)


In [9]:
# ---------------------------
# ➤ Step 1: Select features that actually exist
# ---------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
categorical_cols = ['Product ID', 'Location']
numerical_cols = [
    'quantity_in_stock', 'quantity_sold', 'Shelf Life (days)', 'Days_Until_Expiry', 'demand_velocity'
]

# Encode Product ID and Location
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Final input features
feature_cols = categorical_cols + numerical_cols
X = df[feature_cols].values
y = df[['reorder_quantity', 'days_to_reorder']].values  # Multi-output regression

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np

# Convert to float32 for TabNet
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

# Initialize and train TabNet model
tabnet = TabNetRegressor()

tabnet.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric=['mae'],
    max_epochs=100,
    patience=10,
    batch_size=512,
    virtual_batch_size=128
)


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 10511.95557| val_0_mae: 67.19666290283203|  0:00:00s
epoch 1  | loss: 10146.44629| val_0_mae: 61.12004089355469|  0:00:00s
epoch 2  | loss: 28093.81738| val_0_mae: 59.087440490722656|  0:00:00s
epoch 3  | loss: 11328.60693| val_0_mae: 57.405670166015625|  0:00:00s
epoch 4  | loss: 28918.98291| val_0_mae: 57.081581115722656|  0:00:00s
epoch 5  | loss: 17692.64258| val_0_mae: 56.365699768066406|  0:00:00s
epoch 6  | loss: 37228.51074| val_0_mae: 56.084068298339844|  0:00:00s
epoch 7  | loss: 18077.76221| val_0_mae: 56.25511932373047|  0:00:00s
epoch 8  | loss: 16482.85596| val_0_mae: 56.64337158203125|  0:00:01s
epoch 9  | loss: 28752.52246| val_0_mae: 56.981239318847656|  0:00:01s
epoch 10 | loss: 34908.15894| val_0_mae: 56.9498405456543|  0:00:01s
epoch 11 | loss: 15877.45776| val_0_mae: 56.561798095703125|  0:00:01s
epoch 12 | loss: 35478.43384| val_0_mae: 56.5090217590332|  0:00:01s
epoch 13 | loss: 14517.69141| val_0_mae: 56.44921875|  0:00:01s
epoch 14 | loss: 2625

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Predict
y_pred = tabnet.predict(X_test)

# Metrics
mae_qty = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
rmse_qty = mean_squared_error(y_test[:, 0], y_pred[:, 0], squared=False)

mae_days = mean_absolute_error(y_test[:, 1], y_pred[:, 1])
rmse_days = mean_squared_error(y_test[:, 1], y_pred[:, 1], squared=False)

print(f" MAE - Reorder Quantity: {mae_qty:.2f}")
print(f" RMSE - Reorder Quantity: {rmse_qty:.2f}")
print(f" MAE - Days to Reorder: {mae_days:.2f}")
print(f" RMSE - Days to Reorder: {rmse_days:.2f}")


 MAE - Reorder Quantity: 76.54
 RMSE - Reorder Quantity: 91.88
 MAE - Days to Reorder: 35.63
 RMSE - Days to Reorder: 288.14
